### Create 3D point wise clusters (runtime: ~ 2h)

In [29]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as col
import matplotlib.cm as cm
import numpy as np
import cv2
import os
from tqdm import tqdm

from open3d import *
from nuscenes.nuscenes import NuScenes, NuScenesExplorer
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import view_points
from pyquaternion import Quaternion
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
from mpl_toolkits import mplot3d
from PIL import Image
from PIL import ImageDraw

#### Set paths
<div class="alert alert-block alert-warning">
<h4>ToDo:</h4>
<ol>
    <li> Set "datasets_root" to the root of your <b>dataset directory</b>
    <li> Set "coda_root" to the root of your new <b>CODA directory</b>
</ol>
</div>

In [2]:
datasets_root = f'/disk/ml/datasets/'
coda_root = f'/disk/ml/own_datasets/CODA/'

<div class="alert alert-block alert-warning">
<h4>ToDo:</h4>
<ol>
    <li> Set how much the x-axis is compressed for DBSCAN (0.1)
    <li> Set "eps" for DBSCAN (0.15)
    <li> Set "min_samples" for DBSCAN (6)
    <li> Set "quantile" for mean shift (0.3)
    <li> Set how many points around the center are considered for cluster selection (5)
    <li> Select if all points (not only those in 2d bounding box) should be shown (False)
    <li> Select if every single annotation should be plotted (False) (if True: runtime: ~ 10h)
    <li> Set dpi for the plots for the manual inspection. (200 should be enough, 400 for print quality) (400 dpi: runtime: ~ 4h)
</ol>
</div>

In [40]:
# dbscan
weight_x_lidar = 0.1
eps = 0.15
min_samples = 6

# mean shift
quantile = 0.3

# cluster selection
num_adjacent_points = 5

show_all = False
plot_single_annotations = True
dpi = 400

In [16]:
kitti_path = os.path.join(datasets_root, 'KITTI/object/data')
calib_kitti_path = os.path.join(kitti_path, 'training/calib')

nuscenes_path = os.path.join(datasets_root, 'nuScenes')

once_path = os.path.join(datasets_root, 'ONCE/data_root/data')

lidar_anno_path = os.path.join(coda_root, 'lidar_clustering')
projection_path = os.path.join(coda_root, 'projection_clustering')
projection_anno_path = os.path.join(coda_root, 'projection_clustering_annotation')
image_path = os.path.join(coda_root, 'image')
binary_path = os.path.join(coda_root, 'lidar')
json_nuscenes_indices = os.path.join(coda_root, 'nuscenes_indices.json')
json_corner_case = os.path.join(coda_root, 'corner_case.json')

In [17]:
if not os.path.exists(lidar_anno_path):
        os.makedirs(lidar_anno_path)
if not os.path.exists(projection_path):
        os.makedirs(projection_path)
if not os.path.exists(projection_anno_path):
        os.makedirs(projection_anno_path)

with open(json_nuscenes_indices, 'r') as file:
    nuscenes_indices = json.load(file)
    
with open(json_corner_case, 'r') as file:
    corner_case = json.load(file)
    
images = corner_case['images']
annotations = corner_case['annotations']

#### Load nuScenes

In [ ]:
nusc_trainval = NuScenes(version='v1.0-trainval', dataroot=nuscenes_path, verbose=True)
nusc = NuScenesExplorer(nusc=nusc_trainval)

Save point clouds

In [34]:
def save_points(points, annotation_id):
    points.astype('float32').tofile(os.path.join(lidar_anno_path, str(annotation_id) + '.bin'))

#### Functions to cut the point cloud to frustum size

In [18]:
def cut_normalized_pointcloud(cam, img, name, annotation_bbox, kitti=False, once=False):
    #normalize
    cam = normalize(cam)
    
    # filter point out of canvas
    cam = cut_pointcloud(cam, annotation_bbox, img, once)
    
    
    cam_norm = cam.copy().T
    
    #un_normalize
    cam = un_normalize(cam, kitti)
    
    return cam, cam_norm

def normalize(cam):
    cam[:2] /= cam[2,:]
    return cam

def cut_pointcloud(cam, annotation_bbox, img, once):
    
    u,v,z = cam
    if show_all:
        img = Image.open(img)
        img_w, img_h = img.size
        u_out = np.logical_or(u<0, u>img_w)
        v_out = np.logical_or(v<0, v>img_h)
    else:
        u_out = np.logical_or(u<annotation_bbox[0], u>annotation_bbox[0]+annotation_bbox[2])
        v_out = np.logical_or(v<annotation_bbox[1], v>annotation_bbox[1]+annotation_bbox[3])
    outlier = np.logical_or(u_out, v_out)
    
    return np.delete(cam,np.where(outlier),axis=1)

def un_normalize(cam, kitti):
    cam = cam.T
    for row in cam:
        if not kitti:
            row[0] *= row[2]
            row[1] *= row[2]
        else:
            row[0,0] *= row[0,2]
            row[0,1] *= row[0,2]
    cam = cam.T
    cam = np.insert(cam,3,1,axis=0)
    return cam

#### Functions to plot graph

In [38]:
def plot_graph(cam, img, name, annotations, is_single_annotation=False, is_kitti=False):
    
    u,v,z,c1,c2,c3,c4 = cam
    
    # do projection staff
    img = Image.open(img)
    img_w, img_h = img.size
    
    img_overlay = Image.new('RGB', (img_w, img_h), (255,255,255))
    
    draw = ImageDraw.Draw(img)
    draw_overlay = ImageDraw.Draw(img_overlay)
    
    for annotation in annotations:
        if is_single_annotation:
            anno_id = annotations['id']
            bbox = annotations['bbox']
        else:
            anno_id = annotation['id']
            bbox = annotation['bbox']
        bbox_x = bbox[0]
        bbox_y = bbox[1]
        bbox_w = bbox[2]
        bbox_h = bbox[3]
        
        shape = [(bbox_x, bbox_y), (bbox_x + bbox_w, bbox_y + bbox_h)]
        draw.rectangle(shape, outline = (0,255,0), width = 3)
        draw_overlay.rectangle(shape, fill = (0,255,0), outline = (255,255,255), width = 3)
        result = np.copy(np.array(img))
        result[~np.all(np.array(img_overlay) == 255*np.ones(3), axis=-1)] = 0.9*np.array(img)[~np.all(np.array(img_overlay) == 255*np.ones(3), axis=-1)] + 0.1*np.array(img_overlay)[~np.all(np.array(img_overlay) == 255*np.ones(3), axis=-1)]
        
        result_img = Image.fromarray(result.astype('uint8'))
        
    if is_single_annotation:
        plot_figure(result_img, img_w, img_h, u, v, z, c1, c2, c3, c4, name=f'{anno_id}', save_path=os.path.join(projection_anno_path, str(anno_id)), is_kitti=is_kitti)    
    else:
        plot_figure(result_img, img_w, img_h, u, v, z, c1, c2, c3, c4, name=f'{name}', save_path=os.path.join(projection_path, name), is_kitti=is_kitti)
    

def plot_figure(img, img_w, img_h, u, v, z, c1, c2, c3, c4, name="", save_path="", show_plot=False, is_kitti=False):
    
    fig = plt.figure(figsize=(10,18),dpi=dpi,tight_layout=False)
    ax = fig.add_subplot(411)
    plt.title(name)
    ax.imshow(img)
    np_color = get_normalized_colors(c2, is_kitti)
    ax.scatter([u],[v],c=np_color[:,:3],alpha=1,s=0.5)
    
    ax = fig.add_subplot(412)
    ax.imshow(img)
    np_color = get_normalized_colors(c4, is_kitti)
    ax.scatter([u],[v],c=np_color[:,:3],alpha=1,s=0.5)
    
    ax = fig.add_subplot(425, projection='3d')
    np_color = get_normalized_colors(c1, is_kitti)
    ax.scatter([u],[z],[-v],c=np_color[:,:3],alpha=1,s=0.8)
    
    ax = fig.add_subplot(426, projection='3d')
    np_color = get_normalized_colors(c2, is_kitti)
    ax.scatter([u],[z],[-v],c=np_color[:,:3],alpha=1,s=0.8)
    
    ax = fig.add_subplot(427, projection='3d')
    np_color = get_normalized_colors(c3, is_kitti)
    ax.scatter([u],[z],[-v],c=np_color[:,:3],alpha=1,s=0.8)
    
    ax = fig.add_subplot(428, projection='3d')
    np_color = get_normalized_colors(c4, is_kitti)
    ax.scatter([u],[z],[-v],c=np_color[:,:3],alpha=1,s=0.8)
    
    if save_path:
        
        plt.savefig(f'{save_path}.png',bbox_inches='tight')
    if show_plot:
        plt.show()
    plt.close()
    
def get_normalized_colors(color, is_kitti=False):
    cmap = cm.get_cmap('jet')
    norm = col.Normalize(vmin=np.min(color), vmax=np.max(color))
    np_colors = [cmap(norm(c)) for c in color]
    np_color = np.asarray(np_colors)
    if is_kitti:
        np_color = np_color[0,0]
    return np_color

#### Functions to cluster point cloud

In [20]:
def getLabels(points, cam_norm, center_point):
    
    db_labels = dbscan(points)
    
    points, cam_norm = setLabels(points, cam_norm, center_point, db_labels)
    
    ms_labels = meanshift(points)
    
    points, cam_norm = setLabels(points, cam_norm, center_point, ms_labels)
    
    
    return points, cam_norm

def dbscan(points):
    
    points_weighted = np.copy(points)
    points_weighted[:,2] = weight_x_lidar * points_weighted[:,2]
    
    dbscan = DBSCAN(eps = eps, min_samples = min_samples).fit(points_weighted) # fitting the model
    labels = np.array(dbscan.labels_).reshape(-1,1)
    
    return labels

def meanshift(points):
    
    bandwidth = estimate_bandwidth(points, quantile=quantile)
    if bandwidth == 0.0:
        bandwidth = 1.0
    ms = MeanShift(bandwidth=bandwidth)
    ms.fit(points)
    labels = np.array(ms.labels_).reshape(-1,1)
    
    return labels

def setLabels(points, cam_norm, center_point, labels):
    
    points = np.hstack((points, labels.astype('float32')))
    cam_norm = np.hstack((cam_norm, labels.astype('float32')))
    
    index_closest_point = get_closest_point(center_point, cam_norm)
    label_closest_point = labels.copy()[index_closest_point]
    labels_closest_point = labels.copy()
    
    for i in range(len(labels)):
        if labels[i] == label_closest_point:
            labels_closest_point[i] = 1
        else:
            labels_closest_point[i] = -1
            
    points = np.hstack((points, labels_closest_point.astype('float32')))
    cam_norm = np.hstack((cam_norm, labels_closest_point.astype('float32')))
    
    return points, cam_norm

def get_closest_point(center_point, points):
    
    diff = np.subtract(points[:,:2], center_point)
    distance = np.einsum('ij,ij->i', diff, diff)
    sorted_distance = np.argsort(distance)
    return sorted_distance[np.argmin(points[sorted_distance[:num_adjacent_points],2])]

#### Functions to translate the point clouds into 2D image space, cut to frustum size, and perform the clustering to get the labels.

ONCE

In [21]:
def create_frustum_once(name, img, binary, calib, annotation):
    
    annotation_id = annotation['id']
    annotation_bbox = annotation['bbox']
    
    cam_to_velo = np.array(calib['calib']['cam03']['cam_to_velo'])
    cam_intrinsic = np.array(calib['calib']['cam03']['cam_intrinsic'])
    distortion = np.array(calib['calib']['cam03']['distortion'])
    cam_intrinsic_n, _ = cv2.getOptimalNewCameraMatrix(cam_intrinsic, distortion, (1920, 1020), alpha=0.0, newImgSize=(1920, 1020))
    cam_intrinsic = np.hstack([cam_intrinsic_n, np.zeros((3, 1), dtype=np.float32)])
    
    
    scan = np.fromfile(binary, dtype=np.float32).reshape((-1,4))
    points = scan[:, 0:3]
     
    points = np.insert(points,3,1,axis=1)
    cam = np.dot(points, np.linalg.inv(cam_to_velo).T)
    mask = cam[:,2] > 0
    cam = cam[mask]
    cam = np.dot(cam, cam_intrinsic.T)
    cam = cam.T
    
    
    #cut and normalize pointcloud
    cam, cam_norm = cut_normalized_pointcloud(cam, img, name, annotation_bbox, once=True)
    
    
    center_point = [annotation_bbox[0] + annotation_bbox[2] / 2, annotation_bbox[1] + annotation_bbox[3] / 2]
    
    cam_intrinsic = np.insert(cam_intrinsic,3,values=[0,0,0,1],axis=0)
    points = np.dot(np.linalg.inv(cam_intrinsic), cam)
    points = np.dot(cam_to_velo, points)
    points = np.round(points,3)
    points = points.T
    
    points, cam_norm = getLabels(points[:, :3], cam_norm, center_point)
    
    #save as bin
    save_points(points, annotation_id)
    if plot_single_annotations:
        plot_graph(cam_norm.T, img, name, annotation, is_single_annotation=True)
    
    return cam_norm
    

KITTI

In [22]:
def create_frustum_kitti(name, img, binary, calib, annotation):
    
    annotation_id = annotation['id']
    annotation_bbox = annotation['bbox']
    
    # P2 (3 x 4) for left eye
    P2 = np.matrix([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3,4)
    R0_rect = np.matrix([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3,3)
    # Add a 1 in bottom-right, reshape to 4 x 4
    R0_rect = np.insert(R0_rect,3,values=[0,0,0],axis=0)
    R0_rect = np.insert(R0_rect,3,values=[0,0,0,1],axis=1)
    Tr_velo_to_cam = np.matrix([float(x) for x in calib[5].strip('\n').split(' ')[1:]]).reshape(3,4)
    Tr_velo_to_cam = np.insert(Tr_velo_to_cam,3,values=[0,0,0,1],axis=0)
    Tr_cam_to_velo = np.matrix([float(x) for x in calib[6].strip('\n').split(' ')[1:]]).reshape(3,4)
    Tr_cam_to_velo = np.insert(Tr_cam_to_velo,3,values=[0,0,0,1],axis=0)

    # read raw data from binary
    scan = np.fromfile(binary, dtype=np.float32).reshape((-1,4))
    points = scan[:, 0:3]# lidar xyz (front, left, up)
    points = np.insert(points,3,1,axis=1).T
    points = np.delete(points,np.where(points[0,:]<0),axis=1)
    cam = P2 @ R0_rect @ Tr_velo_to_cam @ points
    cam = np.delete(cam,np.where(cam[2,:]<0)[1],axis=1)
    
    #cut and normalize pointcloud
    cam, cam_norm = cut_normalized_pointcloud(cam, img, name, annotation_bbox, kitti=True)
    
    center_point = [annotation_bbox[0] + annotation_bbox[2] / 2, annotation_bbox[1] + annotation_bbox[3] / 2]
    
    #inverse matrices
    inv_Tr_velo_to_cam = np.linalg.inv(Tr_velo_to_cam)
    inv_R0_rect = np.linalg.inv(R0_rect)
    P2 = np.insert(P2,3,values=[0,0,0,1],axis=0)
    inv_P2 = np.linalg.inv(P2)
    
    points = inv_Tr_velo_to_cam @ inv_R0_rect @ inv_P2 @ cam
    points[3] = 0.0
    points = np.round(points,3)
    points = points.T
    points, cam_norm = getLabels(points[:, :3], cam_norm, center_point)
    #save as bin
    save_points(points, annotation_id)
    if plot_single_annotations:
        plot_graph(cam_norm.T, img, name, annotation, is_single_annotation=True, is_kitti=True)
    
    return cam_norm
    

nuScenes

In [23]:
def create_frustum_nuscenes(name, img, binary, annotation):
    
    annotation_id = annotation['id']
    annotation_bbox = annotation['bbox']
    
    # Get token ('nuscenes_033402.jpg': '1a41ba0751d5497ebd32df7c86950671')
    token_nuscenes = nuscenes_indices[f'nuscenes_{name}.jpg']
    
    my_sample = nusc_trainval.get('sample', token_nuscenes)
    cam = nusc_trainval.get('sample_data', my_sample['data']['CAM_FRONT'])
    pointsensor = nusc_trainval.get('sample_data', my_sample['data']['LIDAR_TOP'])
    
    cs_record_p = nusc_trainval.get('calibrated_sensor', pointsensor['calibrated_sensor_token'])
    poserecordp = nusc_trainval.get('ego_pose', pointsensor['ego_pose_token'])
    poserecordc = nusc_trainval.get('ego_pose', cam['ego_pose_token'])
    cs_record_c = nusc_trainval.get('calibrated_sensor', cam['calibrated_sensor_token'])
    
    
    pc = LidarPointCloud.from_file(binary)
    
    
    pc.rotate(Quaternion(cs_record_p['rotation']).rotation_matrix)
    pc.translate(np.array(cs_record_p['translation']))

    pc.rotate(Quaternion(poserecordp['rotation']).rotation_matrix)
    pc.translate(np.array(poserecordp['translation']))
    
    pc.translate(-np.array(poserecordc['translation']))
    pc.rotate(Quaternion(poserecordc['rotation']).rotation_matrix.T)
    
    pc.translate(-np.array(cs_record_c['translation']))
    pc.rotate(Quaternion(cs_record_c['rotation']).rotation_matrix.T)
    
    cam = view_points(pc.points[:3, :], np.array(cs_record_c['camera_intrinsic']), normalize=False)
    
    cam = np.delete(cam,np.where(cam[2,:]<0)[0],axis=1)
    
    #cut and normalize pointcloud
    cam, cam_norm = cut_normalized_pointcloud(cam, img, name, annotation_bbox)
    
    center_point = [annotation_bbox[0] + annotation_bbox[2] / 2, annotation_bbox[1] + annotation_bbox[3] / 2]
    
    
    view = np.array(cs_record_c['camera_intrinsic'])
    viewpad = np.eye(4)
    viewpad[:view.shape[0], :view.shape[1]] = view
    
    pc.points = np.dot(np.linalg.inv(viewpad), cam)
    
    pc.rotate(Quaternion(cs_record_c['rotation']).rotation_matrix)
    pc.translate(np.array(cs_record_c['translation']))
    
    pc.rotate(Quaternion(poserecordc['rotation']).rotation_matrix)
    pc.translate(np.array(poserecordc['translation']))
    
    pc.translate(-np.array(poserecordp['translation']))
    pc.rotate(Quaternion(poserecordp['rotation']).rotation_matrix.T)
    
    pc.translate(-np.array(cs_record_p['translation']))
    pc.rotate(Quaternion(cs_record_p['rotation']).rotation_matrix.T)
    
    points = pc.points
    points = np.round(points,3)
    points = points.T
    
    if points.size == 0:
        points = np.array([[1.0,1.0,1.0,1.0]])
        cam_norm = np.array([[1.0,1.0,1.0]])
    points, cam_norm = getLabels(points[:, :3], cam_norm, center_point)
    
    #save as bin
    save_points(points, annotation_id)
    if plot_single_annotations:
        plot_graph(cam_norm.T, img, name, annotation, is_single_annotation=True)
    
    return cam_norm

#### Create labeles for each annotation in every image

In [ ]:
for image in tqdm(images):
    id = image['id']
    file_name = image['file_name'].split('.')[0]
    prefix, name = file_name.split('_')
    img = os.path.join(image_path, file_name + '.jpg')
    binary = os.path.join(binary_path, file_name + '.bin')
    annotations_in_image = []
    annotations_in_cam = [-1,-1,-1,-1,-1,-1,-1]
    
    
    for annotation in annotations:
        if(annotation['image_id']) == id:
            annotations_in_image.append(annotation)
    
    if prefix == 'kitti':
        img = img.split('.')[0] + '.png'
        with open(os.path.join(calib_kitti_path, name + '.txt'),'r') as f:
            calib = f.readlines()
        for annotation in annotations_in_image:
            annotations_in_cam = np.vstack([annotations_in_cam, create_frustum_kitti(name, img, binary, calib, annotation)])
        plot_graph(annotations_in_cam.T, img, file_name, annotations_in_image, is_kitti=True)
    elif prefix == 'nuscenes':
        for annotation in annotations_in_image:
            annotations_in_cam = np.vstack([annotations_in_cam, create_frustum_nuscenes(name, img, binary, annotation)])
        plot_graph(annotations_in_cam.T, img, file_name, annotations_in_image)
    else:
        calib = json.load(open(os.path.join(once_path, prefix, prefix + '.json'), 'r'))
        for annotation in annotations_in_image:
            annotations_in_cam = np.vstack([annotations_in_cam, create_frustum_once(name, img, binary, calib, annotation)]) 
        plot_graph(annotations_in_cam.T, img, file_name, annotations_in_image)  